In [ ]:
# requirements.txt 
# 通常用來列出 Python 專案所依賴的所有外部套件及其版本。
# 主要作用是方便開發者在不同的環境中快速重建相同的開發環境，並確保專案所依賴的所有套件都能正確安裝。
# pip install -r requirements.txt

# BERT NLP套件Sentence Transformers，除了提供各種訓練好的BERT模型套件可直接使用，更方便的提供了fine-tune的方法，讓我們可以訓練自己的NLP model。
# （BERT的訓練可分為預訓練pretraining及微調fine-tune兩步，透過fine-tune加上自己標記的資料，便可將預訓練模型train至符合各項業務場景的需求。）
# # !pip install sentence_transformers

In [ ]:
"""
從資料庫中獲取資料 → 載入模型 → 文字轉向量function → 向量處理 → 存入資料庫(ES)

"""

In [4]:
import json
from datetime import datetime
from pathlib import Path
from liontk.enum.es import ES
from liontk.es.es import Elasticsearch
import os
import arrow
import settings
from dotenv import load_dotenv
from elasticsearch import Elasticsearch
from sentence_transformers import SentenceTransformer
from sklearn.preprocessing import normalize

/home/schoenhsiung/.virtualenvs/py38/lib/python3.8/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [ ]:
# 從 es 獲取資料
today = datetime.today().strftime("%Y%m%d")
es = Elasticsearch.get_connect(ES.DEV_DS)
index = f"group-{today.replace('-', '')}"
group_file_name = Path(__file__).parent / "es_group.json"

# 在ipynb中使用
from pathlib import Path
group_file_name = Path.cwd()/ "es_group.json"

In [ ]:
print(f"start to extract data from {index}")
search_body = {"query": {"match_all": {}}}
es_grp_data = es.search(index=index, body=search_body, size=1000, scroll="3m")
sid = es_grp_data["_scroll_id"]
scroll_size = len(es_grp_data["hits"]["hits"])
data_list = []
while scroll_size > 0:
    for contents in es_grp_data.get("hits").get("hits"):
        data_list.append(contents["_source"])
    es_grp_data = es.scroll(scroll_id=sid, scroll="3m")
    sid = es_grp_data["_scroll_id"]
    scroll_size = len(es_grp_data["hits"]["hits"])
    print(f"scroll size : {scroll_size}")

es.close()

start to extract data from group-20250310
scroll size : 1000
scroll size : 1000
scroll size : 1000
scroll size : 1000
scroll size : 1000
scroll size : 889
scroll size : 0


In [16]:
len(data_list)

6889

In [ ]:
# 將資料存成 json
with open(group_file_name, "w") as file:
    file.write(json.dumps(data_list))
print(f"Successfully save data into {group_file_name}")

successfully save data into /home/schoenhsiung/crawler/Z_download/new_grp_rag/es_group.json


In [ ]:
# 向量處理
# process_vector_field

In [26]:
# with open(group_file_name, "r") as file:
#     grp_data = json.load(file)
test_datas = grp_data[5:11]
test_datas
for d in test_datas:
    print('from', d.get('DEPARTURE_LIST', ''))
    print('to', d.get('DESTINATION_LIST', ''))

from [{'CITY_CODE': 'KHH', 'CITY_NAME': '高雄'}]
to [{'CITY_ABBR': '高雄市', 'CITY_ID': '13358', 'CITY_NAME': '高雄市', 'COUNTRY_ABBR': '台灣', 'COUNTRY_ID': '13323', 'COUNTRY_NAME': '中華民國', 'CTO_CITY_ID': '', 'CTO_CITY_NAME': '', 'CTO_COUNTRY_ID': '', 'CTO_COUNTRY_NAME': ''}, {'CITY_ABBR': '台南市', 'CITY_ID': '13457', 'CITY_NAME': '台南市', 'COUNTRY_ABBR': '台灣', 'COUNTRY_ID': '13323', 'COUNTRY_NAME': '中華民國', 'CTO_CITY_ID': '', 'CTO_CITY_NAME': '', 'CTO_COUNTRY_ID': '', 'CTO_COUNTRY_NAME': ''}]
from []
to []
from [{'CITY_CODE': 'TPE', 'CITY_NAME': '台北'}]
to [{'CITY_ABBR': '北海道', 'CITY_ID': '10434', 'CITY_NAME': '北海道', 'COUNTRY_ABBR': '日本', 'COUNTRY_ID': '10406', 'COUNTRY_NAME': '日本國', 'CTO_CITY_ID': '', 'CTO_CITY_NAME': '', 'CTO_COUNTRY_ID': '', 'CTO_COUNTRY_NAME': ''}]
from [{'CITY_CODE': 'TPE', 'CITY_NAME': '台北'}]
to [{'CITY_ABBR': '熊本縣', 'CITY_ID': '11922', 'CITY_NAME': '熊本縣', 'COUNTRY_ABBR': '日本', 'COUNTRY_ID': '10406', 'COUNTRY_NAME': '日本國', 'CTO_CITY_ID': '', 'CTO_CITY_NAME': '', 'CTO_COUNTRY_I

In [10]:
embeding_model = "infgrad/stella-mrl-large-zh-v3.5-1792d"
# model = SentenceTransformer(embeding_model)

In [11]:
def text_to_vector(query_str: str) -> list:
    """Converts a string into an embedding vector"""
    vectors = model.encode([query_str], normalize_embeddings=False) 
    # n_dims 越大效果越好，但時空消耗也越大。建議選取 128 的倍數，因為模型是這樣訓練的
    n_dims = 1792
    cut_vecs = normalize(vectors[:, :n_dims])  # 對向量進行標準化，使數值範圍介於0~1，資料會較符合常態分佈，減小離群值的影響，有助於提高後續操作的穩定性
    return cut_vecs.flatten().tolist()

In [39]:
for data in test_datas:
    # FromVector
    if data.get("DEPARTURE_LIST"):
        from_text = "".join(d.get("CITY_NAME", "") for d in data['DEPARTURE_LIST'])
        if from_text:
            from_vector = text_to_vector(from_text)
        # print('from_text:', from_text)
        # print('from_vector:', from_vector)

    # ToVector
    if data.get("DESTINATION_LIST"):
        if data.get("DESTINATION_LIST"):
            to_text = "".join(d.get("CITY_NAME", "") for d in data["DESTINATION_LIST"])
        else:
            to_text = data.get("AREA_NAME", "") + data.get("AREAD_NAME", "")

        if to_text:
            to_vector = text_to_vector(to_text)
        # print('to_text:', to_text)
        # print('to_vector:', to_vector)

    # FromToVector
    from_to_text = ""
    if data.get("DEPARTURE_LIST"):
        from_to_text += "".join(f"{depart.get('CITY_NAME')}出發" for depart in data["DEPARTURE_LIST"] if depart.get("CITY_NAME"))
    if data.get("DESTINATION_LIST"):
        from_to_text += "".join(f"去 {dest.get('CITY_NAME')}" for dest in data["DESTINATION_LIST"] if dest.get("CITY_NAME"))
    else:
        from_to_text += "去 " + (data.get("AREA_NAME", "") + data.get("AREAD_NAME", ""))

    if from_to_text:
        from_to_vector = text_to_vector(from_to_text.strip())
    # print('from_to_text:', from_to_text)
    # print('from_to_vector:', from_to_vector)

    day = len(data["DAILY_DESC"])
    day_desc = "一日遊" if day == 0 else f"{day}天{day - 1}夜"
    trip_types = "".join(type["NAME"] for type in data.get("TRIP_TYPE_LIST", []))
    mixed_text = f'{from_to_text.strip()}{day_desc}{data["PROD_NAME"]}{data["PROD_ONM"]}{data["PROD_DESC"]}{data["ITINERARY_DESC"]}{trip_types}'
    data["embedding"] = text_to_vector(mixed_text)
    print(mixed_text)
    print(data["embedding"])

高雄出發去 高雄市去 台南市1天0夜台南｜關子嶺水火同源.紅葉公園.雅聞湖濱療癒森林.井仔腳瓦盤鹽田一日｜高雄出發系KG特選南關子嶺散遊1日關子嶺散遊趣逗陣跟著來關子嶺風景區位於白河區東側，是台南著名的旅遊勝地，四周有枕頭山、虎頭山、大凍山、雞籠山等群峰環抱，景色幽靜的風景區周邊沿線有水火同源、火山碧雲寺、東山咖啡、溫泉老街、閑雲橋、火王爺廟、嶺頂公園、寶泉橋、新舊好漢坡步道等休憩玩賞地點，皆是來到關子嶺不可錯過的景點。紅葉公園自紅蜥蜴牌樓拾階而上，沿途飛舞的蝴蝶裝飾，引人入勝。紅葉公園是全台紫斑蝶生態最豐富的地點之一，透過西拉雅國家風景區管理處與保育團體透過生態教育活動進行誘蝶植物栽種，打造適合蝴蝶群聚的友善環境，吸引紫斑蝶及其他各式蝶種在此處棲息，因此紅葉公園為台南市重要的環境教育場所。1.注意蚊蟲叮咬及防曬。2.欄杆處請小心，切勿攀附嬉戲。【井仔腳瓦盤鹽田】井仔腳瓦盤鹽田是北門的第一座鹽田，也是現存最古老的瓦盤鹽田遺址，原為清領時期的瀨東鹽場，西元1818年遷移此至，至今未再移位，是台灣最古老的現址鹽田，至今已有200年歷史。以厚約6公厘的瓦缸片鋪成的瓦盤鹽田，呈現出馬賽克拼貼般的美麗藝術，為延續其曬鹽產業文化的精神，將鹽場復育，讓「井仔腳」特殊的曬鹽景觀保留在北門永華里的海岸邊，現已成為雲嘉南濱海國家風景區內最具特色的觀光鹽田。此地讓人重溫了居民共有的生活回憶，遊客更可體驗傳統曬鹽、挑鹽、收鹽的樂趣，深入了解臺灣製鹽史地的點點滴滴，是一處適合親子同遊的深度旅遊景點。(圖片僅供參考：雲嘉南濱海國家風景區管理處網站提供)餐食安排第一天早餐：敬請自理第一天午餐：甕窯雞風味餐第一天晚餐：敬請自理費用包含【交通】遊覽車1日來回車資及過路橋費（依最終實際人數安排車款）。【餐食】午餐。費用不含:check_mark:隨團領隊人員及司機差旅費每人每天（含孩童佔車）$150*1天，共$150。※個人因素所產生之消費，如飲料、酒類、私人購物費等。※個人旅遊平安保險，依規定旅客若有個別需求，得自行投保旅行平安保險。 ※本行程表上未註明之各項開銷，建議、自費或自由行程所衍生之任何費用。溫馨提醒:check_mark:旅客務必提供一支台灣手機聯絡號碼，以利領隊出團時可與您聯繫集合事宜。※因山區氣候多變，建議準備保暖衣物。並請旅客考量自身健康狀況酌情參加，並配合提供緊急聯絡人資料。※行

In [ ]:
import os

os.environ  # 檢查所有環境變數